In [1]:
!git clone https://github.com/NicolasCambon/RNN_UniversalDependencies_FR.git #Téléchargment des données

fatal: destination path 'RNN_UniversalDependencies_FR' already exists and is not an empty directory.


In [2]:
def distance_verbe(fic):
  try:
    fichier = open(fic,"r")
  except IOError:
    print("le fichier",fic, "n'existe pas")
    return None
  
  liste=[]
  for ligne in fichier :  
    ligne = ligne.strip('\n\r') 
    seq = [x for x in ligne.split()]  
    liste.append(seq)  
  fichier.close()
  
  sentence = []
  part=[]
  for l in range (len(liste)-1):
    if len(liste[l])!= 0:
      part.append(liste[l])
    else:
      sentence.append(part)
      part=[]
  num=0
  deno=0
  for i in range (len(sentence)):
    lis=[]
    for j in range(len(sentence[i])):
      if len(sentence[i][j])>8 and sentence[i][j][7]== 'root':
        chiffre = sentence[i][j][0]
        for k in range(len(sentence[i])):
          if len(sentence[i][k])>8 and sentence[i][k][6]== chiffre and 'nsubj'in sentence[i][k][7] and sentence[i][k][0] not in lis:
            lis.append(sentence[i][k][0])
            num += int(chiffre)-int(sentence[i][k][0])
            deno+=1
  return (num/deno)

print(distance_verbe('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-train.conllu.txt'))
print(distance_verbe('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-test.conllu.txt'))
         

4.508871989860583
4.569131832797428


In [87]:
import numpy as np

def donne(fic):
  print("lecture des donnees d'entrée depuis le fichier : ", fic) 
  try:
    fichier = open(fic,"r")
  except IOError:
    print("le fichier",fic, "n'existe pas")
    return None
  
  liste=[]
  for ligne in fichier :  
    ligne = ligne.strip('\n\r') 
    seq = [x for x in ligne.split()]  
    liste.append(seq)  
  fichier.close()
  
  sentence = []
  part=[]
  for l in range (len(liste)-1):
    if len(liste[l])!= 0:
      part.append(liste[l])
    else:
      sentence.append(part)
      part=[]

  POP=[]
  Out=[]
  for i in range(len(sentence)):  
    pa=[]
    no=[]
    if i <= len(sentence)//2:
      for e in range(2,len(sentence[i])):
        flag=0
        inf=''
        nomb=''
        if sentence[i][e][3] == "AUX":
          if sentence[i][e][2] == "avoir":
            inf+='_avoir'
          else:
            inf+='_etre'

        if "Sing" in sentence[i][e][5]:
          nomb+="_S"
        elif "Plur" in sentence[i][e][5]:
          nomb+="_P"

        pa.append(sentence[i][e][3]+inf+nomb)
        if sentence[i][e][3] != "VERB":
          no.append(3)
        if sentence[i][e][3] == "VERB" and len(nomb) != 0:
          no.append(2)
        if sentence[i][e][3] == "VERB" and len(nomb) == 0:
          no.append(3)
      POP.append(pa)
      Out.append(no)
      
    else:
      for e in range(2,len(sentence[i])):
        flag=0
        inf=''
        nomb=''
        if sentence[i][e][3] == "AUX":
          if sentence[i][e][2] == "avoir":
            inf+='_avoir'
          else:
            inf+='_etre'
            
        if sentence[i][e][3] != "VERB":
          if "Sing" in sentence[i][e][5]:
            nomb+="_S"
          elif "Plur" in sentence[i][e][5]:
            nomb+="_P"
          pa.append(sentence[i][e][3]+inf+nomb)
          no.append(3)
        elif sentence[i][e][3] == "VERB":
          if "Sing" in sentence[i][e][5]:
            nomb+="_P"
          elif "Plur" in sentence[i][e][5]:
            nomb+="_S"
          pa.append(sentence[i][e][3]+inf+nomb)
          if len(nomb) != 0:
            no.append(1)
          else: #elif len(nomb) == 0:
            no.append(3)
           
      POP.append(pa)
      Out.append(no)
  return(POP,Out) 



def maximun(POP_tr,POP_te):
  maxi = len(POP_tr[0])
  for i in POP_tr:
    if len(i) > maxi:
      maxi=len(i)
  for j in POP_te:
    if len(j) > maxi:
      maxi=len(j)
  return(maxi)

def dico(POP_tr,POP_te):
  dic={}
  lis=[]
  k=1
  for i in range (len(POP_tr)):
    for j in range(len(POP_tr[i])):
      if POP_tr[i][j] not in lis:
        lis.append(POP_tr[i][j])
        dic[POP_tr[i][j]]=k
        k+=1
        
  for i in range (len(POP_te)):
    for j in range(len(POP_te[i])):
      if POP_te[i][j] not in lis:
        lis.append(POP_te[i][j])
        dic[POP_te[i][j]]=k
        k+=1  
  return(dic)

def convert(POP,dic):
  Input=[]
  for i in range (len(POP)):
    intp=[]
    for j in range(len(POP[i])):
      intp.append(dic[POP[i][j]])
    Input.append(intp)
  return(Input)


def rempli(POP_tr,POP_te,Out_tr,Out_te,maxi):
  
  for i in range (len(POP_tr)):
    if len(POP_tr[i]) < maxi:
      for k in range(maxi-len(POP_tr[i])):
        POP_tr[i].append(0)
        Out_tr[i].append(0)
  
  for j in range (len(POP_te)):
    if len(POP_te[j]) < maxi:
      for k in range(maxi-len(POP_te[j])):
        POP_te[j].append(0)
        Out_te[j].append(0)
 
 
  return(POP_tr,Out_tr,POP_te,Out_te)


def mise_en_forme(Inp,Out):
  
  POP_r = np.array(Inp).reshape (len(Inp),maxi,1)
  Out_r = np.array(Out).reshape (len(Out),maxi)
  
  return(POP_r,Out_r)



inp_tr,Out_tr= donne('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-train.conllu.txt')
inp_te,Out_te= donne('RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-test.conllu.txt')
maxi = maximun(inp_tr,inp_te)
dictionnaire=dico(inp_tr,inp_te)

x_train = convert(inp_tr,dictionnaire)
x_test = convert(inp_te,dictionnaire)
inp_tr_r,Out_tr_r,inp_te_r,Out_te_r= rempli(x_train,x_test,Out_tr,Out_te,maxi)

x_train,y_train = mise_en_forme(inp_tr_r,Out_tr_r)
print('x shape = ', x_train.shape)
print('y shape = ', y_train.shape)

x_test,y_test = mise_en_forme(inp_te_r,Out_te_r)
print('x shape = ', x_test.shape)
print('y shape = ', y_test.shape)




lecture des donnees d'entrée depuis le fichier :  RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-train.conllu.txt
lecture des donnees d'entrée depuis le fichier :  RNN_UniversalDependencies_FR/Donnees/fr_sequoia-ud-test.conllu.txt
x shape =  (2230, 150, 1)
y shape =  (2230, 150)
x shape =  (455, 150, 1)
y shape =  (455, 150)


In [101]:
Verbe_correct = 0
Verbe_faux = 0
indice_Vc=[]
indice_Vf=[]
for i in range (len(y_train)):
  ivc=[]
  ivf=[]
  for j in range(len(y_train[i])):
    if y_train[i][j] == 1:
      Verbe_faux += 1
      ivf.append(j)
    elif y_train[i][j] == 2:
      Verbe_correct += 1
      ivc.append(j)
  indice_Vc.append(ivc)
  indice_Vf.append(ivf)

print(Verbe_correct)
print(Verbe_faux)

1388
1682


In [103]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed




model = Sequential()  
model.add(LSTM(150,input_shape=(150,1),return_sequences=False,dropout=0.2, recurrent_dropout=0.2))
#model.add(TimeDistributed(Dense(activation='linear',units=1)))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=50,batch_size=100,validation_data=(x_test, y_test))


Train on 2230 samples, validate on 455 samples
Epoch 1/50
2230/2230 [==============================] - 16s 7ms/step - loss: 2.7804 - acc: 0.8305 - val_loss: 1.0848 - val_acc: 0.5578
Epoch 2/50
2230/2230 [==============================] - 12s 5ms/step - loss: 0.1675 - acc: 0.3917 - val_loss: -1.0376 - val_acc: 0.2865
Epoch 3/50
2230/2230 [==============================] - 12s 5ms/step - loss: -1.1081 - acc: 0.3187 - val_loss: -1.0866 - val_acc: 0.3767
Epoch 4/50
2230/2230 [==============================] - 12s 5ms/step - loss: -1.1978 - acc: 0.4259 - val_loss: -1.1320 - val_acc: 0.4484
Epoch 5/50
2230/2230 [==============================] - 12s 5ms/step - loss: -1.2333 - acc: 0.4505 - val_loss: -1.1612 - val_acc: 0.4537
Epoch 6/50
2230/2230 [==============================] - 12s 5ms/step - loss: -1.2741 - acc: 0.4514 - val_loss: -1.1928 - val_acc: 0.4527
Epoch 7/50
2230/2230 [==============================] - 12s 5ms/step - loss: -1.2879 - acc: 0.4527 - val_loss: -1.2430 - val_acc: 0.45

In [104]:
l=model.predict(x_train)
vcnp = 0
vfnp = 0
for i in range(len(l)):
  for j in range(len(l[i])):
    for k in range (len(indice_Vc[i])):
      if indice_Vc[i][k] == j:
        if int(l[i][j]) == 2:
          vcnp +=1
    for p in range (len(indice_Vf[i])):
      if indice_Vf[i][p] == j:
        if int(l[i][j]) == 1:
          vfnp +=1
print(vcnp)
print(vfnp)

0
1256
